In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [11]:
# import cost of living csv
coli_df = pd.read_csv("coli_composite.csv")
coli_df.head()

,Urban Area,State,COLI Composite
0,Anniston-CalhounCounty,AL,87.4
1,Auburn-Opelika,AL,92.3
2,Birmingham,AL,91.2
3,Decatur-Hartselle,AL,85.1
4,Dothan,AL,87.0


In [12]:
# import median rent per square feet by city (zillow)
mrsqft_df = pd.read_csv("median_rents_sqft.csv")
mrsqft_df.head()

,Region Name,State,2018-01
0,New York,NY,1.542
1,Los Angeles,CA,1.956
2,Chicago,IL,1.178
3,Dallas,TX,0.926
4,Philadelphia,PA,1.040


In [13]:
# import median property value per square feet by city (zillow)
mvsqft_df = pd.read_csv("median_value_sqft.csv")
mvsqft_df.head()

,Region Name,State,2018-01
0,New York,NY,255
1,Los Angeles,CA,423
2,Chicago,IL,142
3,Dallas,TX,118
4,Philadelphia,PA,140


In [14]:
# import office build out costs per city (JLL)
office_boc_sqft = pd.read_csv("office_boc_sqft.csv")
office_boc_sqft.head()

,City/Region,State,BOC_Sqft
0,Atlanta,GA,128.83
1,Austin,TX,141.22
2,Boston,MA,156.92
3,Chicago,IL,174.70
4,Columbus,OH,150.06


In [15]:
# import hq2 city list
city_list = pd.read_csv("amazon_hq2_finalists.csv")
city_list.head()

,City/Region,State
0,Atlanta,GA
1,Austin,TX
2,Boston,MA
3,Chicago,IL
4,Columbus,OH


In [16]:
# merging residential real estate costs, office real estate costs, and cost of living data part 1
new_df = pd.merge(city_list, mvsqft_df,  how='left', left_on=['City/Region','State'], right_on = ['Region Name','State'])
del new_df['Region Name']
new_df.columns = ['Region', 'State', 'Median Value / Sqft']
new_df = pd.merge(new_df, mrsqft_df,  how='left', left_on=['Region','State'], right_on = ['Region Name','State'])
del new_df['Region Name']
new_df.columns = ['Region', 'State', 'Median Value / Sqft', 'Median Rent / Sqft']

In [17]:
# merging residential real estate costs, office real estate costs, and cost of living data part 2
merged_df = pd.merge(new_df, coli_df,  how='left', left_on=['Region','State'], right_on = ['Urban Area','State'])
merged_df = pd.merge(merged_df, office_boc_sqft, how='left', left_on=['Region','State'], right_on = ['City/Region','State'])
del merged_df['Urban Area']
del merged_df['City/Region']
merged_df

,Region,State,Median Value / Sqft,Median Rent / Sqft,COLI Composite,BOC_Sqft
0,Atlanta,GA,103,0.782,98.8,128.83
1,Austin,TX,151,0.988,97.3,141.22
2,Boston,MA,271,1.514,147.9,156.92
3,Chicago,IL,142,1.178,123.4,174.70
4,Columbus,OH,114,0.892,89.6,150.06
5,Dallas,TX,118,0.926,101.9,138.00
6,Denver,CO,248,1.382,111.8,143.10
7,Indianapolis,IN,88,0.760,92.4,167.89
8,Los Angeles,CA,423,1.956,147.8,130.68
9,Miami,FL,176,1.382,114.4,140.40


In [18]:
# set variables and rank median house value data
mv_max = max(merged_df['Median Value / Sqft'])
mv_min = min(merged_df['Median Value / Sqft'])
mv_range = mv_max - mv_min
merged_df['mv_score'] = 2.0

for index, row in merged_df.iterrows():
    score = (abs(row['Median Value / Sqft'] - mv_max))/mv_range*10
    score = round(score, 1)
    merged_df.set_value(index, 'mv_score', score)

# set variables and rank median rent data
mr_max = max(merged_df['Median Rent / Sqft'])
mr_min = min(merged_df['Median Rent / Sqft'])
mr_range = mr_max - mr_min
merged_df['mr_score'] = 2.0

for index, row in merged_df.iterrows():
    score = (abs(row['Median Rent / Sqft'] - mr_max))/mr_range*10
    score = round(score, 1)
    merged_df.set_value(index, 'mr_score', score)

# set variables and rank coli data
coli_max = max(merged_df['COLI Composite'])
coli_min = min(merged_df['COLI Composite'])
coli_range = coli_max - coli_min
merged_df['coli_score'] = 2.0

for index, row in merged_df.iterrows():
    score = (abs(row['COLI Composite'] - coli_max))/coli_range*10
    score = round(score, 1)
    merged_df.set_value(index, 'coli_score', score)

# set variables and rank boc data
boc_max = max(merged_df['BOC_Sqft'])
boc_min = min(merged_df['BOC_Sqft'])
boc_range = boc_max - boc_min
merged_df['boc_score'] = 2.0

for index, row in merged_df.iterrows():
    score = (abs(row['BOC_Sqft'] - boc_max))/boc_range*10
    score = round(score, 1)
    merged_df.set_value(index, 'boc_score', score)

merged_df['Total Score'] = (merged_df['mv_score']+merged_df['mr_score']+merged_df['coli_score']+merged_df['boc_score'])/4
merged_df['Total Score'] = round(merged_df['Total Score'], 2)
merged_df.sort_values('Total Score', ascending=False)

,Region,State,Median Value / Sqft,Median Rent / Sqft,COLI Composite,BOC_Sqft,mv_score,mr_score,coli_score,boc_score,Total Score
0,Atlanta,GA,103,0.782,98.8,128.83,9.6,9.8,9.1,6.7,8.80
17,Raleigh,NC,129,0.808,96.2,134.07,8.8,9.6,9.3,6.0,8.43
4,Columbus,OH,114,0.892,89.6,150.06,9.2,8.9,10.0,3.9,8.00
5,Dallas,TX,118,0.926,101.9,138.00,9.1,8.6,8.8,5.5,8.00
7,Indianapolis,IN,88,0.760,92.4,167.89,10.0,10.0,9.7,1.5,7.80
11,Nashville,TN,137,0.920,96.6,148.06,8.5,8.7,9.3,4.1,7.65
16,Pittsburgh,PA,103,0.832,99.4,160.17,9.6,9.4,9.0,2.5,7.62
1,Austin,TX,151,0.988,97.3,141.22,8.1,8.1,9.2,5.0,7.60
15,Philadelphia,PA,140,1.040,117.0,156.85,8.4,7.7,7.2,3.0,6.58
9,Miami,FL,176,1.382,114.4,140.40,7.4,4.8,7.5,5.1,6.20


In [19]:
# import education statistics from National Center for Education Statistics
edu_stats_df = pd.read_csv("education_statistics.csv")
edu_stats_df = edu_stats_df[['State Abb', 'Total']]
edu_stats_df = pd.merge(city_list, edu_stats_df, how='left', left_on=['State'], right_on = ['State Abb'])
del edu_stats_df['State Abb']

# set variables for ranking data
edu_max = max(edu_stats_df['Total'])
edu_min = min(edu_stats_df['Total'])
edu_range = edu_max - edu_min
edu_stats_df['degree_score'] = 2.0

# loop through and create degree score
for index, row in edu_stats_df.iterrows():
    score = (row['Total'] - edu_min)/edu_range*10
    score = round(score, 1)
    edu_stats_df.set_value(index, 'degree_score', score)

edu_stats_df    
edu_stats_df.sort_values('degree_score', ascending=False)

,City/Region,State,Total,degree_score
8,Los Angeles,CA,226411.0,10.0
13,New York,NY,169040.0,7.3
5,Dallas,TX,153102.0,6.5
1,Austin,TX,153102.0,6.5
9,Miami,FL,124331.0,5.2
16,Pittsburgh,PA,117020.0,4.8
15,Philadelphia,PA,117020.0,4.8
3,Chicago,IL,107583.0,4.4
2,Boston,MA,84944.0,3.3
4,Columbus,OH,78498.0,3.0


In [77]:
# import camelot climate index from csv 
cci_df = pd.read_csv("camelot_climate_index.csv")
cci_df.head()

# attempt merge
cci_df_merged = pd.merge(city_list, cci_df, how='left', left_on=['City/Region'], right_on = ['City'])
cci_df_merged = cci_df_merged[['City/Region', 'State_x', 'INDEX']]
cci_df_merged.columns = [['City', 'State', 'Camelot Index']]
cci_df_merged

,City,State,Camelot Index
0,Atlanta,GA,70
1,Austin,TX,66
2,Boston,MA,68
3,Chicago,IL,63
4,Columbus,OH,63
5,Dallas,TX,66
6,Denver,CO,67
7,Indianapolis,IN,64
8,Los Angeles,CA,86
9,Miami,FL,70
